# 번역기 만들기

- https://github.com/nr-ai/Aiffel_Exploration/blob/fee789670470342b318418031035d91e72c1b808/translator_fin.ipynb

##   프랑스어와 영어의 병렬 코퍼스 다운

In [1]:
#! wget https://www.manythings.org/anki/fra-eng.zip

--2020-09-22 17:46:35--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.24.109.196, 104.24.108.196, ...
접속 www.manythings.org (www.manythings.org)|172.67.173.198|:443... 접속됨.
HTTP request sent, awaiting response... 200 OK
Length: 6073837 (5.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   5.79M  2.82MB/s    in 2.1s    

2020-09-22 17:46:38 (2.82 MB/s) - ‘fra-eng.zip’ saved [6073837/6073837]



## library import

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

import re    
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Dropout
from tensorflow.keras.models import Model

## file read

In [2]:
import os
file_path = os.getenv('HOME')+'/aiffel/AIFFEL_LSG/lms/01-EXPLORATION/E15-translator/data/fra-eng/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
28650,Don't do that to me.,Ne me fais pas ça.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
158142,I was told that I needed to get enough sleep.,On m'a dit que j'avais besoin de prendre suffi...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
162123,Tom is someone I really enjoy hanging out with.,Tom est quelqu'un avec qui j'aime vraiment tra...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
157807,How much did you have to pay for the tickets?,Combien t'a-t-il fallu payer pour les billets ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
28358,Can I pay on credit?,Puis-je payer à crédit ?,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


## data 전처리

### 불필요 부분 제거 및 3.3만 데이터만 추출

In [3]:
# cc column  빠드리고 5만게만 데이터 추출
lines = lines[['eng', 'fra']][:33000] # 3.3만개 샘플 사용 
lines.sample(5)

,eng,fra
24047,I can explain this.,Je peux expliquer cela.
17240,We're undressing.,Nous sommes en train de nous déshabiller.
24173,I don't speak fast.,Je ne parle pas vite.
2840,That is all.,C'est tout.
13288,Who punched you?,Qui vous a frappé ?


### 구두점 및 소문자

In [33]:
eng_lines = []
fra_lines = []


sos_token = '<start> '
eos_token = ' <end>'
'

def preprocess_line(line):

    # 소문자 변환
    line = line.lower().strip()
    
    # 구두점(Punctuation)을 단어와 분리하기
    line = re.sub(r"([?.!,¿])", r" \1 ", line)
    line = re.sub(r'[" "]+', " ", line)
    line = re.sub(r"[^a-zA-Z?.!,¿]+", " ", line)

    line = line.strip()
    
    return line

for line in lines.eng:
    v = preprocess_line(line)
    eng_lines.append(v)
    line = v
    
for line in lines.fra:
    v = preprocess_line(sos_token + line + eos_token)
    fra_lines.append(v)
    line = v

fra_lines


SyntaxError: EOL while scanning string literal (<ipython-input-33-df1a8fb3bab6>, line 7)

In [34]:
eng_lines

['go .',
 'hi .',
 'hi .',
 'run !',
 'run !',
 'who ?',
 'wow !',
 'fire !',
 'help !',
 'jump .',
 'stop !',
 'stop !',
 'stop !',
 'wait !',
 'wait !',
 'go on .',
 'go on .',
 'go on .',
 'hello !',
 'hello !',
 'i see .',
 'i try .',
 'i won !',
 'i won !',
 'i won .',
 'oh no !',
 'attack !',
 'attack !',
 'cheers !',
 'cheers !',
 'cheers !',
 'cheers !',
 'get up .',
 'go now .',
 'go now .',
 'go now .',
 'got it !',
 'got it !',
 'got it ?',
 'got it ?',
 'got it ?',
 'hop in .',
 'hop in .',
 'hug me .',
 'hug me .',
 'i fell .',
 'i fell .',
 'i fled .',
 'i know .',
 'i left .',
 'i left .',
 'i lied .',
 'i lost .',
 'i paid .',
 'i m .',
 'i m ok .',
 'i m ok .',
 'listen .',
 'no way !',
 'no way !',
 'no way !',
 'no way !',
 'no way !',
 'no way !',
 'no way !',
 'no way !',
 'no way !',
 'really ?',
 'really ?',
 'really ?',
 'thanks .',
 'thanks .',
 'we try .',
 'we won .',
 'we won .',
 'we won .',
 'we won .',
 'ask tom .',
 'awesome !',
 'be calm .',
 'be calm .

In [35]:
lines.sample(5)

,eng,fra
974,They swam.,Ils nageaient.
30313,I watched the movie.,J'ai vu le film.
30845,It tastes very good.,C’est très bon.
17500,Who wrote Hamlet?,Qui a écrit Hamlet ?
24606,I ran to my mother.,Je courus voir ma mère.


### train test data set split

In [36]:
#lines = lines[['eng', 'fra']][:30000]
#test_data = lines[['eng', 'fra']][30000:33000]

### 토큰화

In [37]:
eng_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(eng_lines)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(eng_lines)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[28], [1129], [1129]]

In [38]:
fra_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(fra_lines)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(fra_lines)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[86], [1063], [1063]]

In [39]:
# 고유한토큰의 갯수

eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4659
프랑스어 단어장의 크기 : 7321


In [40]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 7
프랑스어 시퀀스의 최대 길이 14


In [41]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 7
프랑스어 시퀀스의 최대 길이 14


In [42]:
input_text

[[28],
 [1129],
 [1129],
 [257],
 [257],
 [58],
 [2038],
 [476],
 [70],
 [912],
 [71],
 [71],
 [71],
 [132],
 [132],
 [28, 42],
 [28, 42],
 [28, 42],
 [978],
 [978],
 [1, 72],
 [1, 90],
 [1, 92],
 [1, 92],
 [1, 92],
 [913, 31],
 [979],
 [979],
 [1725],
 [1725],
 [1725],
 [1725],
 [49, 44],
 [28, 53],
 [28, 53],
 [28, 53],
 [64, 3],
 [64, 3],
 [64, 3],
 [64, 3],
 [64, 3],
 [2438, 37],
 [2438, 37],
 [680, 17],
 [680, 17],
 [1, 398],
 [1, 398],
 [1, 3279],
 [1, 51],
 [1, 157],
 [1, 157],
 [1, 519],
 [1, 121],
 [1, 431],
 [1, 11],
 [1, 11, 201],
 [1, 11, 201],
 [384],
 [31, 165],
 [31, 165],
 [31, 165],
 [31, 165],
 [31, 165],
 [31, 165],
 [31, 165],
 [31, 165],
 [31, 165],
 [99],
 [99],
 [99],
 [293],
 [293],
 [8, 90],
 [8, 92],
 [8, 92],
 [8, 92],
 [8, 92],
 [218, 9],
 [377],
 [33, 465],
 [33, 465],
 [33, 465],
 [33, 359],
 [33, 564],
 [33, 564],
 [33, 564],
 [33, 564],
 [33, 564],
 [33, 564],
 [33, 305],
 [33, 117],
 [33, 117],
 [33, 117],
 [33, 117],
 [33, 117],
 [33, 117],
 [774, 3],


In [43]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index['start'] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index['end'] ] for line in target_text]


KeyError: 'start'

In [30]:
fra_tokenizer.word_index

{'je': 1,
 'est': 2,
 'vous': 3,
 'pas': 4,
 'il': 5,
 'a': 6,
 'j': 7,
 'nous': 8,
 'ai': 9,
 'le': 10,
 'de': 11,
 'ne': 12,
 'l': 13,
 'c': 14,
 'tom': 15,
 'tu': 16,
 'suis': 17,
 'd': 18,
 'un': 19,
 'n': 20,
 's': 21,
 't': 22,
 'ce': 23,
 'la': 24,
 'en': 25,
 'me': 26,
 'tes': 27,
 'que': 28,
 'une': 29,
 'es': 30,
 'les': 31,
 'e': 32,
 'm': 33,
 'elle': 34,
 'moi': 35,
 'te': 36,
 'y': 37,
 'sommes': 38,
 'sont': 39,
 'tait': 40,
 'ils': 41,
 'fait': 42,
 'tout': 43,
 'r': 44,
 'mon': 45,
 'qui': 46,
 're': 47,
 'des': 48,
 'elles': 49,
 'tr': 50,
 'aime': 51,
 'as': 52,
 'qu': 53,
 'avons': 54,
 'faire': 55,
 'pr': 56,
 'avez': 57,
 'toi': 58,
 'ici': 59,
 'peux': 60,
 'bien': 61,
 'personne': 62,
 'besoin': 63,
 'on': 64,
 'veux': 65,
 'cela': 66,
 'votre': 67,
 'du': 68,
 'tre': 69,
 'ma': 70,
 'arr': 71,
 'se': 72,
 'ont': 73,
 'tais': 74,
 'comment': 75,
 'ton': 76,
 'aller': 77,
 'tous': 78,
 'faut': 79,
 'pour': 80,
 'air': 81,
 'fais': 82,
 'trop': 83,
 'maintenant': 

In [26]:
print(decoder_input[:3])
print(decoder_target[:3])

NameError: name 'decoder_input' is not defined

In [13]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23)
프랑스어 입력데이터의 크기(shape) : (50000, 76)
프랑스어 출력데이터의 크기(shape) : (50000, 76)


In [14]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [15]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23, 51)
프랑스어 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 76, 73)


In [16]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 23, 51)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 76, 73)


## model

In [21]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

### 인코더 설계

In [20]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

### 디코더 설계

In [22]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [23]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

### 모델 셋팅

In [24]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 51)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 315392      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]            

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
368/368 [==============================] - 7s 19ms/step - loss: 0.9245 - val_loss: 0.7914
Epoch 2/50
368/368 [==============================] - 6s 16ms/step - loss: 0.5697 - val_loss: 0.6574
Epoch 3/50
368/368 [==============================] - 6s 16ms/step - loss: 0.4734 - val_loss: 0.5718
Epoch 4/50
368/368 [==============================] - 6s 16ms/step - loss: 0.4160 - val_loss: 0.5235
Epoch 5/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3767 - val_loss: 0.4830
Epoch 6/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3477 - val_loss: 0.4485
Epoch 7/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3259 - val_loss: 0.4341
Epoch 8/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3086 - val_loss: 0.4177
Epoch 9/50
368/368 [==============================] - 6s 16ms/step - loss: 0.2947 - val_loss: 0.4080
Epoch 10/50
368/368 [==============================] - 6s 16ms/step - loss: 0.2829 - val_lo

## model test

In [ ]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

### 디코더 설계

In [ ]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [ ]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

In [ ]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [45]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [44]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

NameError: name 'decode_sequence' is not defined